In [1]:
# installing the libraries for transformers
#!pip install -U -q sentence-transformers transformers bitsandbytes accelerate sentencepiece

# install libarary to convert latex to text
#!pip install -U pylatexenc


In [2]:
""" # test tokenize latex
from transformers import AlbertTokenizer
import torch

# Load the tokenizer
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

# Example LaTeX math expressions
latex_expressions = [
    r"\frac{a}{b}",
    r"\int_0^{\infty} e^{-x^2} dx",
    r"E = mc^2"
]

# Tokenize the LaTeX expressions
tokens = tokenizer(latex_expressions, padding=True, truncation=True, return_tensors="pt")

# Print tokenized input
print(tokens.input_ids)
 """

<>:1: SyntaxWarning: invalid escape sequence '\i'
<>:1: SyntaxWarning: invalid escape sequence '\i'
C:\Users\billl\AppData\Local\Temp\ipykernel_9796\354891771.py:1: SyntaxWarning: invalid escape sequence '\i'
  """ # test tokenize latex


' # test tokenize latex\nfrom transformers import AlbertTokenizer\nimport torch\n\n# Load the tokenizer\ntokenizer = AlbertTokenizer.from_pretrained(\'albert-base-v2\')\n\n# Example LaTeX math expressions\nlatex_expressions = [\n    r"\x0crac{a}{b}",\n    r"\\int_0^{\\infty} e^{-x^2} dx",\n    r"E = mc^2"\n]\n\n# Tokenize the LaTeX expressions\ntokens = tokenizer(latex_expressions, padding=True, truncation=True, return_tensors="pt")\n\n# Print tokenized input\nprint(tokens.input_ids)\n '

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Deep Learning library
import torch

# to load transformer models
#from sentence_transformers import SentenceTransformer
#from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline # T5 is Google

# to split the data
from sklearn.model_selection import train_test_split

# to compute performance metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score


In [4]:

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
files = []
# on vs code
data_dir = './eedi-mining/eedi-mining-misconceptions-in-mathematics'
miscon_file_index = 0
train_file_index = 3
test_file_index = 2
# on kaggle
# data_dir = '/kaggle/input'
#miscon_file_index = 1
#train_file_index = 2
#test_file_index = 3
for dirname, _, filenames in os.walk(data_dir):
    print (filenames)
    for filename in filenames:
        print(os.path.join(dirname, filename))
        files.append(os.path.join(dirname, filename))
misconceptions_filename = files[miscon_file_index]
train_filename = files[train_file_index]
test_filename = files[test_file_index]
# For Kaggle, You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

['misconception_mapping.csv', 'sample_submission.csv', 'test.csv', 'train.csv']
./eedi-mining/eedi-mining-misconceptions-in-mathematics\misconception_mapping.csv
./eedi-mining/eedi-mining-misconceptions-in-mathematics\sample_submission.csv
./eedi-mining/eedi-mining-misconceptions-in-mathematics\test.csv
./eedi-mining/eedi-mining-misconceptions-in-mathematics\train.csv


## File and Field Information
### [train/test].csv

* QuestionId - Unique question identifier (int).
* ConstructId - Unique construct identifier (int) .
* ConstructName - Most granular level of knowledge related to question (str).
* CorrectAnswer - A, B, C or D (char).
* SubjectId - Unique subject identifier (int).
* SubjectName - More general context than the construct (str).
* QuestionText - Question text extracted from the question image using human-in-the-loop OCR (str) .
* Answer[A/B/C/D]Text - Answer option A text extracted from the question image using human-in-the-loop OCR (str).
* Misconception[A/B/C/D]Id - Unique misconception identifier (int). Ground truth labels in train.csv; your task is to predict these labels for test.csv.

### misconception_mapping.csv
maps MisconceptionId to its MisconceptionName

### sample_submission.csv
A submission file in the correct format.
* QuestionId_Answer - Each question has three incorrect answers for which need you predict the MisconceptionId.
* MisconceptionId - You can predict up to 25 values, space delimited.

# get training data

In [5]:
train_data = pd.read_csv(train_filename)
train_data_orig = train_data.copy()
train_data.head()

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,NaN,NaN,NaN,1672.0
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,2142.0,143.0,2142.0,NaN
2,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,1287.0,NaN,1287.0,1073.0
3,3,2377,Recall and use the intersecting diagonals prop...,88,Properties of Quadrilaterals,C,The angles highlighted on this rectangle with ...,acute,obtuse,\( 90^{\circ} \),Not enough information,1180.0,1180.0,NaN,1180.0
4,4,3387,Substitute positive integer values into formul...,67,Substitution into Formula,A,The equation \( f=3 r^{2}+3 \) is used to find...,\( 30 \),\( 27 \),\( 51 \),\( 24 \),NaN,NaN,NaN,1818.0


In [6]:
print ("train_data shape")
print("rows",train_data.shape[0])
print("columns",train_data.shape[1])

train_data shape
rows 1869
columns 15


## Get test data

In [7]:
test_data = pd.read_csv(test_filename)
test_data_orig = test_data.copy()
test_data.head()
print ("test_data shape")
print("rows", test_data.shape[0])
print("columns",test_data.shape[1])

test_data shape
rows 3
columns 11


## Get misconception data


In [8]:
miscon_data = pd.read_csv(misconceptions_filename)
miscon_data.head()
print ("miscon_data shape")
print("rows", miscon_data.shape[0])
print("columns",miscon_data.shape[1])

miscon_data shape
rows 2587
columns 2


In [9]:
subject_groups = train_data.groupby(["SubjectId","SubjectName"]).count().sort_values('QuestionId',ascending=False).reset_index()
print(subject_groups.shape[0], " different subject groups")
subject_groups.head()

163  different subject groups


,SubjectId,SubjectName,QuestionId,ConstructId,ConstructName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId
0,64,Linear Equations,53,53,53,53,53,53,53,53,53,23,32,30,27
1,171,Linear Sequences (nth term),44,44,44,44,44,44,44,44,44,28,20,21,19
2,33,BIDMAS,37,37,37,37,37,37,37,37,37,20,21,23,24
3,65,Quadratic Equations,36,36,36,36,36,36,36,36,36,18,17,25,14
4,75,Area of Simple Shapes,36,36,36,36,36,36,36,36,36,22,26,23,24


In [10]:
print("percentages of null values")
pd.DataFrame({'Count':train_data.isnull().sum()[train_data.isnull().sum()>0],'Percentage':(train_data.isnull().sum()[train_data.isnull().sum()>0]/train_data.shape[0])*100})

percentages of null values


,Count,Percentage
MisconceptionAId,734,39.272338
MisconceptionBId,751,40.181915
MisconceptionCId,789,42.215088
MisconceptionDId,832,44.515784


## Clean up missing data

In [11]:
# records where more than one misconception id is missing
missing_a_misconception = train_data[(train_data.CorrectAnswer == "A") & (train_data.MisconceptionBId.isnull() | train_data.MisconceptionCId.isnull()  | train_data.MisconceptionDId.isnull()) ]
print(missing_a_misconception.shape)
missing_a_misconception.head()
train_data[(train_data.CorrectAnswer == "A")].shape

(244, 15)


(482, 15)

In [12]:
missing_b_misconpception = train_data[(train_data.CorrectAnswer == "B") & (train_data.MisconceptionAId.isnull() | train_data.MisconceptionCId.isnull()) ]
missing_b_misconpception.head()

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId
10,10,1414,Expand products of three binomials in the form...,335,Expanding Triple Brackets and more,B,John is expanding these three brackets:\n\(\n(...,\( +6 x^{2} \),\( \begin{array}{l}+6 x^{2} \\ -3 x^{2} \\ +5 ...,\( \begin{array}{l}+6 x^{2} \\ +3 x^{2} \\ +5 ...,\( \begin{array}{l}+6 x^{2} \\ -3 x^{2} \\ -5 ...,NaN,NaN,583.0,NaN
13,13,3065,Convert from hours to minutes,209,Time,B,Hannah's journey to camp is \( 3 \) hours and ...,\( 196 \) minutes,\( 18 \) minutes,\( 102 \) minutes,\( 12 \) minutes,NaN,NaN,161.0,NaN
18,18,261,Carry out missing number subtraction problems ...,211,Adding and Subtracting Negative Numbers,B,![Number line with -12 and -7 marked. Starting...,\( -7 \),\( -5 \),\( -2 \),\( -6 \),2179.0,NaN,NaN,1824.0
20,20,2306,"Given the volume of a cuboid, work out missing...",189,Volume of Prisms,B,The volume of this cuboid is \( 30 \mathrm{~cm...,\( 6 \mathrm{~cm} \),\( 5 \mathrm{~cm} \),\( 4 \mathrm{~cm} \),\( 25 \mathrm{~cm} \),NaN,NaN,NaN,1984.0
21,21,1434,Factorise a quadratic expression in the form a...,53,Factorising into a Double Bracket,B,Step 1: Factorise the following expression\n\n...,\( (3 x+2)(3 x+1) \),\( (3 x+2)(x+1) \),Cannot be factorised,\( (3 x+1)(x+2) \),2240.0,NaN,NaN,NaN


In [13]:
misc_con_data = pd.read_csv(misconceptions_filename)
misc_con_data.head()

,MisconceptionId,MisconceptionName
0,0,Does not know that angles in a triangle sum to...
1,1,Uses dividing fractions method for multiplying...
2,2,Believes there are 100 degrees in a full turn
3,3,Thinks a quadratic without a non variable term...
4,4,Believes addition of terms and powers of terms...


## Update data to prepare for creating vectors


In [14]:
# clean up question
# train_data['CleanQuestion'] = train_data['QuestionText'].replace('\n',' ',regex=True)

# create new data frame with just misconceptions
# Create a new empty DataFrame
miscon_model = pd.DataFrame()


In [15]:
# convert latext formatting to text
from pylatexenc.latex2text import LatexNodes2Text
import pylatexenc
# Convert LaTeX to plain text
# Function to convert LaTeX to text
def latex_to_text(latex_string):
    return LatexNodes2Text().latex_to_text(latex_string)

# repolace names with variables to get consistency on vectors
import spacy

# Load the spaCy model
# for running locally, need to do
# python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

# Function to replace all detected names with unique variables
def replace_names_with_variables(text):
    doc = nlp(text)
    name_counter = 1
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            var_name = f"NAME{name_counter}"
            text = text.replace(ent.text, var_name)
            name_counter += 1
    return text

#def clean_math_text(text):
    # clean_text = latex_to_text(text)
    # clean_text = replace_names_with_variables(clean_text)
    # remove new line and other characters
    # print('before',clean_text)
    #clean_text = clean_text.replace('\n',' ')
    # print('after', clean_text)
    
    #return clean_text 
import re

def clean_math_text(text):
    # Lowercase
    text = text.lower()
    # Remove special characters (keeping LaTeX symbols)
    text = re.sub(r'[^a-zA-Z0-9\s\\{}^_]', '', text)
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Example text
text = "Here's an example: \int_0^{\infty} e^{-x^2} dx.\n Let's clean it up!"

# Clean the text
cleaned_text = clean_math_text(text)
print(cleaned_text)


# remove special characters like new line
#miscon_model['subjectName'] = train_data.SubjectName
#miscon_model['constructName'] = train_data.ConstructName
#miscon_model['question'] = train_data['QuestionText'].apply(clean_math_text)
#  Creating the flattened DataFrame
import math
flattened_data = []
for index, row in train_data.iterrows():
    # put incorrect answer A into row
    # print ("a id",row.MisconceptionAId)

    if ((row.CorrectAnswer != "A") & (math.isnan(row.MisconceptionAId) == False)):
        misconception_row = miscon_data[(miscon_data.MisconceptionId==row.MisconceptionAId)]
        formatted_answer = clean_math_text(row['AnswerAText'])
        # flattened_data.append({'subjectName': row['SubjectName'], 'constructName': row['ConstructName'], 'latex_formatted_question': row['QuestionText'], 'question': clean_math_text(row['QuestionText']), 'latex_formatted_wrongAnswer': row['AnswerAText'], 'wrongAnswer': clean_math_text(row['AnswerAText']), 'misconception': misconception_row.MisconceptionName})
    if ((row.CorrectAnswer != "B") & (math.isnan(row.MisconceptionBId)==False)):
        formatted_answer= clean_math_text(row['AnswerBText'])
        # flattened_data.append({'subjectName': row['SubjectName']}, 'constructName': row['ConstructName'], 'latex_formatted_question': row['QuestionText'], 'question': clean_math_text(row['QuestionText']), 'latext_formatted_wrongAnswer': row['AnswerBText'], 'wrongAnswer': clean_math_text(row['AnswerBText']), 'misconception': misconception_row.MisconceptionName})
    if ((row.CorrectAnswer != "C") & (math.isnan(row.MisconceptionCId) ==False)):
        misconception_row = miscon_data[(miscon_data.MisconceptionId==row.MisconceptionCId)]
        formatted_answer = clean_math_text(row['AnswerCText'])        
        # flattened_data.append({'subjectName': row['SubjectName'], 'constructName': row['ConstructName'], 'latex_formatted_question': row['QuestionText'], 'question': clean_math_text(row['QuestionText']), 'latext_formatted_wrongAnswer': row['AnswerCText'], 'wrongAnswer': clean_math_text(row['AnswerCText']), 'misconception': misconception_row.MisconceptionName})
    if ((row.CorrectAnswer != "D") & (math.isnan(row.MisconceptionDId) == False)):
        misconception_row = miscon_data[(miscon_data.MisconceptionId==row.MisconceptionDId)]
        formatted_answer =  clean_math_text(row['AnswerDText'])
        # flattened_data.append({'subjectName': row['SubjectName'], 'constructName': row['ConstructName'], 'latex_formatted_question': row['QuestionText'], 'question': clean_math_text(row['QuestionText']), 'latext_formatted_wrongAnswer': row['AnswerDText'], 'wrongAnswer': clean_math_text(row['AnswerDText']), 'misconception': misconception_row.MisconceptionName})

    flattened_data.append({'text': f"{row['SubjectName']} {row['ConstructName']} {clean_math_text(row['QuestionText'])} {formatted_answer}", "label": misconception_row.MisconceptionName})

flattened_df = pd.DataFrame(flattened_data)


<>:49: SyntaxWarning: invalid escape sequence '\i'
<>:49: SyntaxWarning: invalid escape sequence '\i'
C:\Users\billl\AppData\Local\Temp\ipykernel_9796\1611341085.py:49: SyntaxWarning: invalid escape sequence '\i'
  text = "Here's an example: \int_0^{\infty} e^{-x^2} dx.\n Let's clean it up!"


heres an example \int_0^{\infty} e^{x^2} dx lets clean it up


In [16]:
flattened_df.head()

,text,label
0,BIDMAS Use the order of operations to carry ou...,"1672 Confuses the order of operations, beli..."
1,Simplifying Algebraic Fractions Simplify an al...,2142 Does not know that to factorise a quad...
2,Range and Interquartile Range from a List of D...,1073 Believes if you add the same value to ...
3,Properties of Quadrilaterals Recall and use th...,1180 Does not know the properties of a rect...
4,Substitution into Formula Substitute positive ...,1818 Thinks you can find missing values in ...


In [17]:
# save flattened data
flattened_df.to_csv('./eedi-mining/model/flattened_misconceptions.csv', index=False)

In [18]:
# defining a function to compute the cosine similarity between two embedding vectors
def cosine_score(text):
    # encoding the text
    embeddings = model.encode(text)

    # calculating the L2 norm of the embedding vector
    norm1 = np.linalg.norm(embeddings[0])
    norm2 = np.linalg.norm(embeddings[1])

    # computing the cosine similarity
    cosine_similarity_score = ((np.dot(embeddings[0],embeddings[1]))/(norm1*norm2))

    return cosine_similarity_score

In [ ]:
from transformers import Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset

# Load the tokenizer and model
# Load model directly
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("AnReu/math_pretrained_bert")
model = AutoModelForPreTraining.from_pretrained("AnReu/math_pretrained_bert")

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True)
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
# get data
model_dir = 'c:/ai_ml/kaggle/eedi-mining/model'
#           'C:\ai_ml\kaggle\eedi-mining\model\flattened_misconceptions.csv'
model_data=f"{model_dir}/flattened_misconceptions.csv"
out_dir = model_dir

# Prepare your dataset (replace with your own data)
texts = flattened_df['text'].to_list()
labels =flattened_df['label'].to_list()
dataset = CustomDataset(texts, labels)